# Baseline solution for seismic facies identification challenge

### Architecture: U-Net
### Language: Tensorflow/keras

In [5]:
import tensorflow as tf
import keras as K

from keras.layers import Input, BatchNormalization, Dropout, Dense, Activation
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate


### Define Network architecture

In [3]:
# defines convolutional block

def conv2d_block(input_tensor,n_filters, kernel_size=3, batchnorm=True):
    x = Conv2D(n_filters, (kernel_size,kernel_size),padding='same',\
               kernel_initializer='he_normal')(input_tensor)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    x = Conv2D(n_filters, (kernel_size, kernel_size), padding='same',\
               kernel_initializer='he_normal')(input_tensor)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    return x

# defines unet architecture

def unet(img_input, out_channel n_filters=64, dropout=0.1, k_size=3, batchnorm = True):
    c1 = conv2d_block(img_input, n_filters*1, kernel_size=k_size, batchnorm=batchnorm)
    p1 = MaxPooling2D((2,2))(c1)
    
    c2 = conv2d_block(p1, n_filters*2, kernel_size=k_size, batchnorm=batchnorm)
    p2 = MaxPooling2D((2,2))(c2)
    
    c3 = conv2d_block(p2, n_filters*4, kernel_size=k_size, batchnorm=batchnorm)
    p3 = MaxPooling2D((2,2))(c3)
    
    c4 = conv2d_block(p3, n_filters*8, kernel_size=k_size, batchnorm=batchnorm)
    p4 = MaxPooling2D((2,2))(c4)
    
    c5 = conv2d_block(p4, n_filters*16, batchnorm=batchnorm)
    
    
    
    u6 = Conv2DTranspose(n_filters*8, (k_size,k_size), strides = (2,2), padding='same')(c5)
    u6 = concatenate([u6,c4])
    c6 = conv2d_block(u6, n_filters*8, kernel_size=k_size, batchnorm=batchnorm)
    
    u7 = Conv2DTranspose(n_filters*4, (k_size,k_size),strides = (2,2), padding='same')(c6)
    u7 = concatenate([u7,c3])
    c7 = conv2d_block(u7, n_filters*4, kernel_size=k_size, batchnorm=batchnorm)
    
    u8 = Conv2DTranspose(n_filters*2, (k_size,k_size), strides = (2,2), padding='same')(c7)
    u8 = concatenate([u8,c2])
    c8 = conv2d_block(u8, n_filters*2, kernel_size=k_size, batchnorm=batchnorm)
    
    u9 = Conv2DTranspose(n_filters*1, (k_size,k_size), strides = (2,2), padding='same')(c8)
    u9 = concatenate([u9,c1])
    c9 = conv2d_block(u9, n_filters*1, kernel_size=k_size, batchnorm=batchnorm)
    
    outputs = Conv2D(out_channel, (1,1), activation='softmax')(c9)
    model = Model(inputs= [img_input], outputs= [outputs], name ='U-Net')
    
    return model

## Data pre-processing